# Introduction to SQLite with Jupyter Notebook

Welcome! In this tutorial, we'll explore the basics of working with databases using SQLite. SQLite is a lightweight database that's perfect for learning, and we'll be using some prehistoric creatures as our example data.

By the end of this tutorial, we'll be able to:
- Create database tables
- Insert and query data
- Filter and sort our results
- Add our own data using an interactive form

Let's get started!

## Setting Up Our Environment

First, let's install and load the tools we'll need. The `ipython-sql` package allows us to write SQL directly in our notebook cells using "magic commands" (commands that start with %).

In [ ]:
# Install the required packages
!pip install ipython-sql sqlalchemy -q

In [ ]:
# Load the SQL extension
%load_ext sql

In [ ]:
# Connect to a SQLite database (this will create a new database file)
%sql sqlite:///prehistoric_creatures.db

## Creating Our First Table: Dinosaurs

Now let's create our first table to store information about dinosaurs. We'll use the `%%sql` magic command at the beginning of a cell to tell Jupyter that the entire cell contains SQL code.

Our dinosaur table will have:
- An ID number (primary key)
- Name
- Diet (herbivore, carnivore, or omnivore)
- Length in meters
- Period when they lived

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS dinosaurs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    diet TEXT NOT NULL,
    length_meters REAL,
    period TEXT
);

## Adding Dinosaur Data

Let's populate our dinosaurs table with some interesting specimens. We'll add several different types to give us good data to work with.

In [ ]:
%%sql
INSERT INTO dinosaurs (name, diet, length_meters, period) VALUES
    ('Tyrannosaurus Rex', 'Carnivore', 12.3, 'Late Cretaceous'),
    ('Triceratops', 'Herbivore', 9.0, 'Late Cretaceous'),
    ('Velociraptor', 'Carnivore', 2.0, 'Late Cretaceous'),
    ('Brachiosaurus', 'Herbivore', 25.0, 'Late Jurassic'),
    ('Stegosaurus', 'Herbivore', 9.0, 'Late Jurassic'),
    ('Allosaurus', 'Carnivore', 9.7, 'Late Jurassic'),
    ('Diplodocus', 'Herbivore', 27.0, 'Late Jurassic'),
    ('Ankylosaurus', 'Herbivore', 6.25, 'Late Cretaceous'),
    ('Spinosaurus', 'Carnivore', 15.0, 'Cretaceous'),
    ('Parasaurolophus', 'Herbivore', 10.0, 'Late Cretaceous');

## Creating Our Second Table: Sea Creatures

Now let's create a second table for prehistoric marine life. These creatures lived in the oceans alongside the dinosaurs.

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS sea_creatures (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    diet TEXT NOT NULL,
    length_meters REAL,
    period TEXT
);

## Adding Sea Creature Data

Let's add some fascinating marine reptiles and other sea-dwelling creatures from prehistoric times.

In [ ]:
%%sql
INSERT INTO sea_creatures (name, diet, length_meters, period) VALUES
    ('Mosasaurus', 'Carnivore', 17.0, 'Late Cretaceous'),
    ('Plesiosaurus', 'Carnivore', 3.5, 'Early Jurassic'),
    ('Ichthyosaurus', 'Carnivore', 2.0, 'Early Jurassic'),
    ('Elasmosaurus', 'Carnivore', 14.0, 'Late Cretaceous'),
    ('Liopleurodon', 'Carnivore', 7.0, 'Middle Jurassic'),
    ('Kronosaurus', 'Carnivore', 10.0, 'Early Cretaceous'),
    ('Tylosaurus', 'Carnivore', 14.0, 'Late Cretaceous'),
    ('Shonisaurus', 'Carnivore', 15.0, 'Late Triassic');

## Querying Our Data

Now that we have data in our tables, let's learn how to retrieve it! We'll start with simple queries and gradually make them more complex.

### Viewing All Dinosaurs

The SELECT statement is how we retrieve data. Let's look at all our dinosaurs:

In [ ]:
%%sql
SELECT * FROM dinosaurs;

### Viewing All Sea Creatures

Let's do the same for our marine friends:

In [ ]:
%%sql
SELECT * FROM sea_creatures;

## Filtering Data with WHERE

Often we don't want ALL the data - we want to find specific records. Let's use the WHERE clause to filter our results.

### Finding Carnivorous Dinosaurs

Let's find all the meat-eating dinosaurs:

In [ ]:
%%sql
SELECT name, length_meters, period 
FROM dinosaurs 
WHERE diet = 'Carnivore';

### Finding Large Creatures

Let's find all dinosaurs that were longer than 10 meters:

In [ ]:
%%sql
SELECT name, length_meters, diet
FROM dinosaurs
WHERE length_meters > 10
ORDER BY length_meters DESC;

### Finding Creatures from Specific Time Periods

Let's see which creatures lived during the Late Cretaceous period:

In [ ]:
%%sql
SELECT name, 'Dinosaur' as type, diet, length_meters
FROM dinosaurs
WHERE period = 'Late Cretaceous'
UNION
SELECT name, 'Sea Creature' as type, diet, length_meters
FROM sea_creatures
WHERE period = 'Late Cretaceous'
ORDER BY length_meters DESC;

## Aggregate Functions

SQL has powerful functions that let us calculate statistics about our data. Let's explore some of these.

### Counting Records

How many dinosaurs do we have in our database?

In [ ]:
%%sql
SELECT COUNT(*) as total_dinosaurs FROM dinosaurs;

### Average, Minimum, and Maximum

Let's calculate some statistics about dinosaur sizes:

In [ ]:
%%sql
SELECT 
    AVG(length_meters) as average_length,
    MIN(length_meters) as smallest,
    MAX(length_meters) as largest
FROM dinosaurs;

### Grouping Data

We can group our results by categories. Let's count how many dinosaurs of each diet type we have:

In [ ]:
%%sql
SELECT diet, COUNT(*) as count, AVG(length_meters) as average_length
FROM dinosaurs
GROUP BY diet;

## Comparing Land and Sea

Let's compare the average sizes of our dinosaurs versus sea creatures:

In [ ]:
%%sql
SELECT 'Dinosaurs' as creature_type, 
       COUNT(*) as count,
       AVG(length_meters) as avg_length,
       MAX(length_meters) as max_length
FROM dinosaurs
UNION
SELECT 'Sea Creatures' as creature_type,
       COUNT(*) as count,
       AVG(length_meters) as avg_length,
       MAX(length_meters) as max_length
FROM sea_creatures;

## Interactive Form: Add Your Own Creature!

Now let's create an interactive form that allows us to add our own creatures to either table. This is a great way to practice inserting data into a database.

In [ ]:
# First, let's import the widgets we'll need
import ipywidgets as widgets
from IPython.display import display, clear_output
import sqlite3

# Create a connection to our database
conn = sqlite3.connect('prehistoric_creatures.db')
cursor = conn.cursor()

# Create the form widgets
creature_type = widgets.Dropdown(
    options=['Dinosaur', 'Sea Creature'],
    value='Dinosaur',
    description='Type:'
)

name_input = widgets.Text(
    value='',
    placeholder='Enter creature name',
    description='Name:'
)

diet_input = widgets.Dropdown(
    options=['Carnivore', 'Herbivore', 'Omnivore'],
    value='Carnivore',
    description='Diet:'
)

length_input = widgets.FloatText(
    value=5.0,
    description='Length (m):'
)

period_input = widgets.Text(
    value='',
    placeholder='e.g., Late Cretaceous',
    description='Period:'
)

submit_button = widgets.Button(
    description='Add Creature',
    button_style='success'
)

output_area = widgets.Output()

def add_creature(b):
    with output_area:
        clear_output()
        
        # Get the values from the form
        name = name_input.value
        diet = diet_input.value
        length = length_input.value
        period = period_input.value
        table = 'dinosaurs' if creature_type.value == 'Dinosaur' else 'sea_creatures'
        
        # Validate input
        if not name or not period:
            print("Please fill in all fields!")
            return
        
        try:
            # Insert the new creature
            cursor.execute(f"""
                INSERT INTO {table} (name, diet, length_meters, period)
                VALUES (?, ?, ?, ?)
            """, (name, diet, length, period))
            conn.commit()
            
            print(f"Successfully added {name} to the {table} table!")
            
            # Show the updated table
            cursor.execute(f"SELECT * FROM {table} ORDER BY id DESC LIMIT 5")
            results = cursor.fetchall()
            print(f"\nLast 5 entries in {table}:")
            for row in results:
                print(f"  {row[1]} - {row[2]}, {row[3]}m, {row[4]}")
            
            # Clear the form
            name_input.value = ''
            period_input.value = ''
            
        except Exception as e:
            print(f"Error adding creature: {e}")

submit_button.on_click(add_creature)

# Display the form
print("Add a New Creature to Our Database")
print("===================================\n")
display(creature_type, name_input, diet_input, length_input, period_input, submit_button, output_area)

## Viewing Your Additions

After adding some creatures with the form above, we can run these queries to see our complete updated tables:

In [ ]:
%%sql
SELECT * FROM dinosaurs ORDER BY id DESC LIMIT 10;

In [ ]:
%%sql
SELECT * FROM sea_creatures ORDER BY id DESC LIMIT 10;

## Practice Exercises

Now that we've learned the basics, let's try some practice queries. Try writing SQL to answer these questions:

1. Find all herbivore dinosaurs from the Late Jurassic period
2. Find the three largest sea creatures
3. Count how many creatures (both dinosaurs and sea creatures) lived in each period
4. Find all creatures (land or sea) that are between 5 and 10 meters long

Use the empty cells below to write your queries:

In [ ]:
%%sql
-- Exercise 1: Your query here


In [ ]:
%%sql
-- Exercise 2: Your query here


In [ ]:
%%sql
-- Exercise 3: Your query here


In [ ]:
%%sql
-- Exercise 4: Your query here


## Summary

Great work! In this tutorial, we've learned how to:

- Set up SQLite in a Jupyter notebook using cell magic
- Create tables with appropriate data types
- Insert data into our tables
- Query data using SELECT statements
- Filter results with WHERE clauses
- Sort results with ORDER BY
- Use aggregate functions like COUNT, AVG, MIN, and MAX
- Group data with GROUP BY
- Combine results from multiple tables with UNION
- Create an interactive form to add data

These are the fundamental building blocks of working with databases. As we continue learning, we'll discover more advanced topics like joins, subqueries, and database optimization.

Feel free to experiment more with the database - try adding more creatures, writing different queries, or even creating new tables!

## Clean Up (Optional)

If we want to start fresh, we can drop our tables with these commands:

In [ ]:
# Uncomment these lines if you want to delete the tables and start over
# %%sql
# DROP TABLE IF EXISTS dinosaurs;
# DROP TABLE IF EXISTS sea_creatures;